# Functions and Imports - no pyiron dependence

In [1]:
from h5io_browser.base import read_dict_from_hdf, list_hdf, write_dict_to_hdf
import os

In [2]:
from pyiron_io import convert_lammps_job_dict

In [3]:
def get_node_from_job_dict(job_dict, node):
    node_name_lst = node.split("/")
    tmp_dict = job_dict
    for group in node_name_lst:
        tmp_dict = tmp_dict[group]
    return tmp_dict

# Create LAMMPS job

In [4]:
from pyiron_atomistics import Project

In [5]:
pr = Project("test")

In [6]:
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [7]:
structure = pr.create.structure.ase.bulk("Al", cubic=True)

In [8]:
structure.set_repeat([9,9,9])

/tmp/ipykernel_834/461746984.py:1: DeprecationWarning: pyiron_atomistics.atomistics.structure.atoms.set_repeat is deprecated: Use Atoms.repeat.
  structure.set_repeat([9,9,9])


In [9]:
len(structure)

2916

In [10]:
job = pr.create.job.Lammps("lmp")

In [11]:
job.structure =structure

In [12]:
job.potential = '2002--Mishin-Y--Ni-Al--LAMMPS--ipr1'

In [13]:
job.calc_md(n_ionic_steps=1000, n_print=10, temperature=500.0)

In [14]:
job.run()

The job lmp was saved and received the ID: 1


# Read job dictionary

In [15]:
job_dict = read_dict_from_hdf(
    file_name=job.project_hdf5.file_name,
    h5_path="/",
    recursive=True,
    slash='ignore',
)

# Dataclasses

In [16]:
convert_lammps_job_dict(job_dict[job.job_name])

LammpsJob(calculation_input=LammpsInput(generic_dict=GenericDict(restart_file_list=[], restart_file_dict={}, exclude_nodes_hdf=[], exclude_groups_hdf=[]), interactive=Interactive(interactive_flush_frequency=10000, interactive_write_frequency=10000), generic=GenericInput(calc_mode='md', structure='atoms', fix_symmetry=None, k_mesh_spacing=None, k_mesh_center_shift=None, reduce_kpoint_symmetry=None, restart_for_band_structure=None, path_name=None, n_path=None, fix_spin_constraint=None, max_iter=None, temperature=500.0, n_ionic_steps=1000, n_print=10, temperature_damping_timescale=100.0, pressure_damping_timescale=1000.0, time_step=1), structure=Structure(dimension=3, indices=array([0, 0, 0, ..., 0, 0, 0]), info={}, positions=array([[ 0.   ,  0.   ,  0.   ],
       [ 0.   ,  2.025,  2.025],
       [ 2.025,  0.   ,  2.025],
       ...,
       [32.4  , 34.425, 34.425],
       [34.425, 32.4  , 34.425],
       [34.425, 34.425, 32.4  ]]), species=['Al'], cell=Cell(cell=array([[36.45,  0.  ,  0

# Get LAMMPS schema

In [17]:
file_name = job.project_hdf5.file_name
file_name

'/home/jovyan/test/lmp.h5'

In [18]:
node_lst = list_hdf(file_name=file_name, h5_path="/", recursive=True)[0]
node_lst

['/lmp/DICT_VERSION',
 '/lmp/HDF_VERSION',
 '/lmp/NAME',
 '/lmp/OBJECT',
 '/lmp/TYPE',
 '/lmp/VERSION',
 '/lmp/executable',
 '/lmp/job_id',
 '/lmp/server',
 '/lmp/status',
 '/lmp/input/generic_dict',
 '/lmp/input/interactive',
 '/lmp/input/control_inp/DICT_VERSION',
 '/lmp/input/control_inp/NAME',
 '/lmp/input/control_inp/OBJECT',
 '/lmp/input/control_inp/TYPE',
 '/lmp/input/control_inp/VERSION',
 '/lmp/input/control_inp/data_dict',
 '/lmp/input/generic/DICT_VERSION',
 '/lmp/input/generic/NAME',
 '/lmp/input/generic/OBJECT',
 '/lmp/input/generic/TYPE',
 '/lmp/input/generic/VERSION',
 '/lmp/input/generic/data_dict',
 '/lmp/input/potential_inp/DICT_VERSION',
 '/lmp/input/potential_inp/NAME',
 '/lmp/input/potential_inp/OBJECT',
 '/lmp/input/potential_inp/TYPE',
 '/lmp/input/potential_inp/VERSION',
 '/lmp/input/potential_inp/data_dict',
 '/lmp/input/potential_inp/potential/Citations',
 '/lmp/input/potential_inp/potential/Config',
 '/lmp/input/potential_inp/potential/Filename',
 '/lmp/input

# Write LAMMPS job

In [19]:
node_dict = {node: get_node_from_job_dict(job_dict=job_dict, node=node[1:]) for node in node_lst}

In [20]:
os.remove(file_name)

In [21]:
write_dict_to_hdf(file_name=file_name, data_dict=node_dict)

# Reload job

In [22]:
job = pr.load(job.job_name)

In [23]:
job.input.control

,Parameter,Value,Comment
0,units,metal,
1,dimension,3,
2,boundary,p p p,
3,atom_style,atomic,
4,read_data,structure.inp,
5,include,potential.inp,
6,fix___ensemble,all nvt temp 500.0 500.0 0.1,
7,variable___dumptime,equal 10,
8,variable___thermotime,equal 10,
9,timestep,0.001,


In [24]:
job.input.potential

,Parameter,Value,Comment
0,pair_style,eam/alloy,
1,pair_coeff,* * NiAl02.eam.alloy Ni Al,


In [25]:
job.output.energy_tot

array([-9427.29630088, -9430.14696684, -9432.82349157, -9433.14423094,
       -9432.46197988, -9431.11779442, -9429.84887403, -9428.59786876,
       -9427.38813359, -9426.70956719, -9426.59728248, -9426.49789307,
       -9426.12272411, -9425.59921798, -9425.20912449, -9424.90209584,
       -9424.49520497, -9423.99938741, -9423.57006459, -9423.24848256,
       -9422.96494608, -9422.73886581, -9422.67498343, -9422.7595396 ,
       -9422.83422591, -9422.80728218, -9422.74305075, -9422.77967415,
       -9422.94528544, -9423.13258663, -9423.2946694 , -9423.5107131 ,
       -9423.84002756, -9424.21163482, -9424.54837222, -9424.86419284,
       -9425.20921977, -9425.60463137, -9426.04263383, -9426.51076414,
       -9427.00943396, -9427.53249825, -9428.0367887 , -9428.46758844,
       -9428.83561118, -9429.21325813, -9429.67512814, -9430.21492958,
       -9430.73337493, -9431.12233644, -9431.37229576, -9431.54923363,
       -9431.70507951, -9431.83867285, -9431.96559155, -9432.12935211,
      